In [1]:
import wntr
import wntr.network.controls as controls

In [3]:
inp_file_path = "../Demand_patterns/ctown_map_with_controls.inp"
wn = wntr.network.WaterNetworkModel(inp_file_path)

C:\ProgramData\Anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning:

Not all curves were used in "../Demand_patterns/ctown_map_with_controls.inp"; added with type None, units conversion left to user



In [121]:
control_list = []
control_dict = {}
for control in wn.controls():
    control_dict['tank_tag'] = control[1].condition.name.split(":")[0]
    control_dict['operator'] = control[1].condition.name.split("level")[1][0:2]
    control_dict['value'] = control[1].condition.name.split("=")[1]
    control_dict['actuator_tag'] = str(control[1].actions()[0].target()[0])
    control_dict['actuator_value'] =str(control[1].actions()[0]).split("to")[1].strip()
    control_list.append(control_dict)

{'tank_tag': 'T1', 'operator': '<=', 'value': '4.0', 'actuator_tag': 'PU1', 'actuator_value': 'Open'}
{'tank_tag': 'T1', 'operator': '>=', 'value': '6.3', 'actuator_tag': 'PU1', 'actuator_value': 'Closed'}
{'tank_tag': 'T1', 'operator': '<=', 'value': '1.0', 'actuator_tag': 'PU2', 'actuator_value': 'Open'}
{'tank_tag': 'T1', 'operator': '>=', 'value': '4.5', 'actuator_tag': 'PU2', 'actuator_value': 'Closed'}
{'tank_tag': 'T2', 'operator': '<=', 'value': '0.5', 'actuator_tag': 'V2', 'actuator_value': 'Open'}
{'tank_tag': 'T2', 'operator': '>=', 'value': '5.5', 'actuator_tag': 'V2', 'actuator_value': 'Closed'}
{'tank_tag': 'T3', 'operator': '<=', 'value': '3.0', 'actuator_tag': 'PU4', 'actuator_value': 'Open'}
{'tank_tag': 'T3', 'operator': '>=', 'value': '5.3', 'actuator_tag': 'PU4', 'actuator_value': 'Closed'}
{'tank_tag': 'T3', 'operator': '<=', 'value': '1.0', 'actuator_tag': 'PU5', 'actuator_value': 'Open'}
{'tank_tag': 'T3', 'operator': '>=', 'value': '3.5', 'actuator_tag': 'PU5', 

In [122]:
cpa_file_path = "../Demand_patterns/ctown.cpa"